In [11]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
# over_sampling คือการทำค่า fail 1284 ให้เท่ากับค่า pass 271k เพื่อให้ data ในการ train เพิ่มมากขึ้น
from collections import Counter
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import SMOTE
from collections import Counter
import pandas as pd
import numpy as np
from flask import Flask, jsonify
import json
# from Database import create_sql_connection
# from Database import create_sql_Component_Master
import seaborn as sns
import numpy as np
import base64
import datetime
from flask import send_file, Flask, jsonify, request
from flask_cors import CORS
import pyodbc
import math
import matplotlib.pyplot as plt
import pandas as pd
import pymssql
import matplotlib
matplotlib.use('Agg')

In [12]:
def create_sql_connection():
    try:
        conn = pyodbc.connect(
            'DRIVER={SQL Server};'
            'SERVER=192.168.101.219;'
            'DATABASE=DataforAnalysis;'
            'UID=DATALYZER;'
            'PWD=NMB54321'
        )
        
        return conn
    except pyodbc.Error as e:
        print("Error connecting to SQL Server:", e)
        return None

In [13]:
def fetch_data(model, line, start, end, selecteKPOV, selecteKPIV):
    conn = create_sql_connection()
    cursor = conn.cursor()

    # Define the SQL query with placeholders for parameters
    query_template = """
        SELECT {selecteKPOV}, {selecteKPIV}
        FROM [Diecast].[dbo].[Pivot]
        JOIN [TransportData].[dbo].[Matching_Auto_Unit1] ON [Pivot].Diecast_S_N = [Matching_Auto_Unit1].Barcode_Base
        JOIN [DataforAnalysis].[dbo].[DataML_Test] ON [DataML_Test].Barcode_motor = [Matching_Auto_Unit1].Barcode_Motor
        WHERE [DataML_Test].[Model] = ? 
        AND [DataML_Test].[Line] = ? 
        AND [DataML_Test].[Date] BETWEEN ? AND ?
        AND {conditions}
    """

    # Ensure selecteKPIV is a list
    if not isinstance(selecteKPIV, list):
        selecteKPIV = [selecteKPIV]

    # Create a comma-separated string of selecteKPIV for the SQL query
    selecteKPIV_str = ', '.join(selecteKPIV)

    # Create a list of conditions for each column in selecteKPIV
    conditions = []
    for column in selecteKPIV:
        conditions.append(f"{column} IS NOT NULL")

    # Combine the conditions using 'AND' and wrap them in parentheses
    conditions_str = " AND ".join(conditions)
    conditions_str = f"{conditions_str}"

    # Execute the query with parameters and fetch data
    query = query_template.format(selecteKPOV=selecteKPOV, selecteKPIV=selecteKPIV_str, conditions=conditions_str)
    datasets = pd.read_sql(query, conn, params=(model, line, start, end))

    return datasets

In [14]:
def make_chartML(model, Line, start, end, selecteKPOV, selecteKPIV):
    datasets = fetch_data(model, Line, start, end, selecteKPOV, selecteKPIV)
    corrmat = datasets.corr()

    f, ax = plt.subplots(figsize=(12, 9))
    sns.heatmap(corrmat, vmax=.8, square=True, cmap='coolwarm', annot=True)

    plt.savefig('../TrainingNodeJS/chart/heatmap.png')
    plt.close()

    return send_file(f"../TrainingNodeJS/chart/heatmap.png", mimetype='image/png')



In [15]:
def pairplot(model, Line, start, end, selecteKPOV, selecteKPIV):
    datasets = fetch_data(model, Line, start, end, selecteKPOV, selecteKPIV)
    pairplot = sns.pairplot(datasets, height=4)

    pairplot.savefig('../TrainingNodeJS/chart/pairplot.png')
    plt.close()

    return send_file(f"../TrainingNodeJS/chart/pairplot.png", mimetype='image/png')



In [16]:
def summary_describe(model, Line, start, end, selecteKPOV, selecteKPIV):
    datasets = fetch_data(model, Line, start, end, selecteKPOV, selecteKPIV)
    summary = datasets.describe(include='all')

    # Convert the summary to a JSON format
    summary_json = summary.to_json()
    print(summary_json)

    # If you want to return the JSON data
    return summary_json


In [17]:
def BIN_KPOV(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV):
    # Call the summary_describe function
    data = fetch_data(model, line, start, end, selecteKPOV, selecteKPIV)
    print( f"BIN_KPOV_fetch_data",{selecteKPIV})

    # Modify the selected column as needed
    data[selecteKPOV] = pd.cut(
        data[selecteKPOV],
        bins=[-np.inf, float(minKPOV), float(maxKPOV), np.inf],
        labels=['fail_low', 'Pass', 'fail_high']
    )

    # Replace 'fail_low' and 'fail_high' with 'fail'
    data[selecteKPOV].replace(['fail_low', 'fail_high'], 'fail', inplace=True)

    # Count the occurrences of different values in the original data
    count_before_smote = dict(Counter(data[selecteKPOV]))

    k = 3
    X = data.loc[:, data.columns != selecteKPOV]  # Use selecteKPOV here
    y = data[selecteKPOV]  # Use selecteKPOV here

    # Sampling strategy
    sm = SMOTE(sampling_strategy='minority', k_neighbors=k, random_state=100)
    X_res, y_res = sm.fit_resample(X, y)

    # Concatenate the resampled data
    datasets = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)

    # Replace 'fail_low' and 'fail_high' with 'fail' in the resampled data
    datasets[selecteKPOV].replace(['fail_low', 'fail_high'], 'fail', inplace=True)

    # Count the occurrences of different values in the resampled data
    count_after_smote = dict(Counter(datasets[selecteKPOV]))

    # Convert the DataFrame to a JSON object
    # json_data = datasets.to_json(orient='records')

    # Create a dictionary to include the count results in the JSON response
    response_data = {
        # "data": json_data,
        "count_before_smote": count_before_smote,
        "count_after_smote": count_after_smote
      
    }
    print(datasets)
    return response_data,datasets
    


In [18]:
def data_bin(model, selecteKPIV):
    conn = create_sql_connection()
    cursor = conn.cursor()
    # สร้างรายการเพื่อเก็บ DataFrames สำหรับแต่ละ selecteKPIV
    result_datasets = []
    query = ""  # สร้างตัวแปร query ที่ระดับภายนอกของลูป for
    for KPIV in selecteKPIV:
        query_template = f"""
        with set1 as (SELECT [id]
          ,[Fullname]
          ,[Model]
          ,[Parameter]
          ,[USL]
          ,[LSL]
          ,CL
          ,[USL]-[LSL] as "X"
          ,([USL]-[LSL])/6 as "Y"
          ,[Part]
          ,[Machine]
          ,[empNumber]
          ,[createdAt]
          ,[updatedAt]
      FROM [Component_Master].[dbo].[Master_matchings]
      )
      select 
        [Model]
        ,[Parameter]
        ,[LSL] - 0.0001 as "LCL_3"
        ,[LSL] + 1 * y - 0.0001 as "LCL_2"
        ,[LSL] + 2 * y - 0.0001 as "LCL_1"
        ,[LSL] + 3 * y - 0.0001 as "CL"
        ,[LSL] + 4 * y - 0.0001 as "UCL_1"
        ,[LSL] + 5 * y - 0.0001 as "UCL_2"
        ,[LSL] + 6 * y as "UCL_3"
      from set1
      where Model='{model}' and parameter = '{KPIV}'  -- ใช้ KPIV ที่เลือก
        """
        query = query_template  # กำหนดค่าให้กับตัวแปร query
        datasets_bin = pd.read_sql(query, conn)
        result_datasets.append(datasets_bin)
        print(f"datasets_bin_query", query)
    # รวม DataFrames ในรายการเป็น DataFrame เดียวกัน
    combined_dataframe = pd.concat(result_datasets, axis=0, ignore_index=True)

    # แปลง DataFrame เป็น JSON
    combined_json = combined_dataframe.to_json(orient='records')

    # ส่ง JSON ผ่าน API response
    return jsonify(combined_json),result_datasets

In [19]:
def DATASETS_BIN(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV):
    conn = create_sql_connection()
    cursor = conn.cursor()
    
    for KPIV in selecteKPIV:
        datasets = BIN_KPOV(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV)
        result_datasets = data_bin(model, selecteKPIV)
    # Call summary_describe to get summary information for the current KPIV
        LCL_3 = result_datasets[KPIV]['LCL_3']
        LCL_2 = result_datasets[KPIV]['LCL_2']
        LCL_1 = result_datasets[KPIV]['LCL_1']
        CL = result_datasets[KPIV]['CL']
        UCL_1 = result_datasets[KPIV]['UCL_1']
        UCL_2 = result_datasets[KPIV]['UCL_2']
        UCL_3 = result_datasets[KPIV]['UCL_3']

    # Print the statistics for the current KPIV
    print(f"{KPIV} Statistics:")
    print(f"3LCL: {LCL_3}")
    print(f"2LCL: {LCL_2}")
    print(f"LCL: {LCL_1}")
    print(f"CL: {CL}")
    print(f"UCL: {UCL_1}")
    print(f"2UCL: {UCL_2}")
    print(f"3UCL: {UCL_3}")

    # Modify the corresponding column in the DataFrame
    column_name = f'{KPIV}'  # Use KPIV as the new column name
    datasets[column_name] = pd.cut(
    pd.to_numeric(datasets[column_name], errors='coerce'),
  
        bins=[-np.inf,LCL_3, LCL_2, LCL_1, CL, UCL_1, UCL_2, UCL_3, np.inf],
        labels=['-4', '-3', '-2', '-1', '1', '2', '3', '4'],
        duplicates='drop'
    )

    response_data_json = json.dumps(response_data)
    response_data = {
    "data": response_data_json,

    }
    return response_data_json,response_data


In [20]:
def data_bin(model, selecteKPIV):
    conn = create_sql_connection()
    cursor = conn.cursor()
    print(f"data_bin",selecteKPIV)
    # สร้างรายการเพื่อเก็บ DataFrames สำหรับแต่ละ selecteKPIV
    
    result_datasets = []
    query = ""  # สร้างตัวแปร query ที่ระดับภายนอกของลูป for
    for KPIV in selecteKPIV:
            query_template = f"""
        with set1 as (SELECT [id]
          ,[Fullname]
          ,[Model]
          ,[Parameter]
          ,[USL]
          ,[LSL]
          ,CL
          ,[USL]-[LSL] as "X"
          ,([USL]-[LSL])/6 as "Y"
          ,[Part]
          ,[Machine]
          ,[empNumber]
          ,[createdAt]
          ,[updatedAt]
      FROM [Component_Master].[dbo].[Master_matchings]
      )
      select 
        [Model]
        ,[Parameter]
        ,[LSL] - 0.0001 as "LCL_3"
        ,[LSL] + 1 * y - 0.0001 as "LCL_2"
        ,[LSL] + 2 * y - 0.0001 as "LCL_1"
        ,[LSL] + 3 * y - 0.0001 as "CL"
        ,[LSL] + 4 * y - 0.0001 as "UCL_1"
        ,[LSL] + 5 * y - 0.0001 as "UCL_2"
        ,[LSL] + 6 * y as "UCL_3"
      from set1
      where Model='{model}' and parameter = '{KPIV}'  -- ใช้ KPIV ที่เลือก
        """
        
            query = query_template  # กำหนดค่าให้กับตัวแปร query
            datasets_bin = pd.read_sql(query, conn)
            result_datasets.append(datasets_bin)
            print(f"datasets_bin_query", query)
    # รวม DataFrames ในรายการเป็น DataFrame เดียวกัน
            combined_dataframe = pd.concat(result_datasets, axis=0, ignore_index=True)

    # แปลง DataFrame เป็น JSON
            combined_json = combined_dataframe.to_json(orient='records')

    # ส่ง JSON ผ่าน API response
    return jsonify(combined_json),result_datasets



In [21]:

def data_bin(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV):
    conn = create_sql_connection()
    cursor = conn.cursor()
    
    # สร้างรายการเพื่อเก็บ DataFrames สำหรับแต่ละ selecteKPIV
    result_datasets = []
    
    for KPIV in selecteKPIV:
        query_template = f"""
        with set1 as (SELECT [id]
          ,[Fullname]
          ,[Model]
          ,[Parameter]
          ,[USL]
          ,[LSL]
          ,CL
          ,[USL]-[LSL] as "X"
          ,([USL]-[LSL])/6 as "Y"
          ,[Part]
          ,[Machine]
          ,[empNumber]
          ,[createdAt]
          ,[updatedAt]
      FROM [Component_Master].[dbo].[Master_matchings]
      )
      select 
        [Model]
        ,[Parameter]
        ,[LSL] - 0.0001 as "LCL_3"
        ,[LSL] + 1 * y - 0.0001 as "LCL_2"
        ,[LSL] + 2 * y - 0.0001 as "LCL_1"
        ,[LSL] + 3 * y - 0.0001 as "CL"
        ,[LSL] + 4 * y - 0.0001 as "UCL_1"
        ,[LSL] + 5 * y - 0.0001 as "UCL_2"
        ,[LSL] + 6 * y as "UCL_3"
      from set1
      where Model='{model}' and parameter = '{KPIV}'
        """
        
        query = query_template
        datasets_bin = pd.read_sql(query, conn)
        result_datasets.append(datasets_bin)

    for KPIV in selecteKPIV:
    # Call summary_describe to get summary information for the current KPIV
        LCL_3 = datasets_bin[KPIV]['LCL_3']
        LCL_2 = datasets_bin[KPIV]['LCL_2']
        LCL_1 = datasets_bin[KPIV]['LCL_1']
        CL = datasets_bin[KPIV]['CL']
        UCL_1 = datasets_bin[KPIV]['UCL_1']
        UCL_2 = datasets_bin[KPIV]['UCL_2']
        UCL_3 = datasets_bin[KPIV]['UCL_3']

    # Print the statistics for the current KPIV
    print(f"{KPIV} Statistics:")
    print(f"3LCL: {LCL_3}")
    print(f"2LCL: {LCL_2}")
    print(f"LCL: {LCL_1}")
    print(f"CL: {CL}")
    print(f"UCL: {UCL_1}")
    print(f"2UCL: {UCL_2}")
    print(f"3UCL: {UCL_3}")

    # Modify the corresponding column in the DataFrame
    column_name = f'{KPIV}'  # Use KPIV as the new column name
    result_datasets[column_name] = pd.cut(
    pd.to_numeric(result_datasets[column_name], errors='coerce'),
  
        bins=[-np.inf,LCL_3, LCL_2, LCL_1, CL, UCL_1, UCL_2, UCL_3, np.inf],
        labels=['-4', '-3', '-2', '-1', '1', '2', '3', '4'],
        duplicates='drop'
    )

    response_data_json = json.dumps(response_data)
    response_data = {
    "data": response_data_json,

    }
    return response_data_json,response_data


In [22]:
def BIN_KPIV(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV):
    data = BIN_KPOV(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV)
    response_data = {}   
    for KPIV in selecteKPIV:
    # Call summary_describe to get summary information for the current KPIV
        datasets_bin = data_bin(model)
        
    LCL_3 = datasets_bin[KPIV]['LCL_3']
    LCL_2 = datasets_bin[KPIV]['LCL_2']
    LCL_1 = datasets_bin[KPIV]['LCL_1']
    CL = datasets_bin[KPIV]['CL']
    UCL_1 = datasets_bin[KPIV]['UCL_1']
    UCL_2 = datasets_bin[KPIV]['UCL_2']
    UCL_3 = datasets_bin[KPIV]['UCL_3']

    # Print the statistics for the current KPIV
    print(f"{KPIV} Statistics:")
    print(f"3LCL: {LCL_3}")
    print(f"2LCL: {LCL_2}")
    print(f"LCL: {LCL_1}")
    print(f"CL: {CL}")
    print(f"UCL: {UCL_1}")
    print(f"2UCL: {UCL_2}")
    print(f"3UCL: {UCL_3}")

    # Modify the corresponding column in the DataFrame
    column_name = f'{KPIV}'  # Use KPIV as the new column name
    data[column_name] = pd.cut(
        pd.to_numeric(data[column_name], errors='coerce'),
        bins=[-np.inf, LCL_3, LCL_2, LCL_1, CL, UCL_1, UCL_2, UCL_3, np.inf],
        labels=['-3', '-2', '-1', '0', '1', '2', '3', '4'],
        duplicates='drop'
    )

    # Store the statistics in response_data
    response_data[KPIV] = {
        "3LCL": LCL_3,
        "2LCL": LCL_2,
        "LCL": LCL_1,
        "CL": CL,
        "UCL": UCL_1,
        "2UCL": UCL_2,
        "3UCL": UCL_3
    }
    print(response_data)
    response_data_json = json.dumps(response_data)
    response_data = {
    "data": response_data_json,

    }
    return response_data_json,response_data
    
# Call BIN_KPOV_KPIV with the provided parameters and preprocessed_data
result_datasets, response_data_json, response_data = BIN_KPIV(
    'LONGSP', '3-6', '2023-08-01', '2023-09-14', 'Projection1',
    ['Parallelism_Stack', 'Set_Dimension_Stack','P1_Attractive_1'], 0.4648, 0.5664
)


C:\Users\IT\AppData\Local\Temp\ipykernel_14404\1780064111.py:35: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datasets = pd.read_sql(query, conn, params=(model, line, start, end))


DatabaseError: Execution failed on sql '
        SELECT Projection1, Parallelism_Stack, Set_Dimension_Stack, P1_Attractive_1
        FROM [Diecast].[dbo].[Pivot]
        JOIN [TransportData].[dbo].[Matching_Auto_Unit1] ON [Pivot].Diecast_S_N = [Matching_Auto_Unit1].Barcode_Base
        JOIN [DataforAnalysis].[dbo].[DataML_Test] ON [DataML_Test].Barcode_motor = [Matching_Auto_Unit1].Barcode_Motor
        WHERE [DataML_Test].[Model] = ? 
        AND [DataML_Test].[Line] = ? 
        AND [DataML_Test].[Date] BETWEEN ? AND ?
        AND Parallelism_Stack IS NOT NULL AND Set_Dimension_Stack IS NOT NULL AND P1_Attractive_1 IS NOT NULL
    ': ('42S22', "[42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'Parallelism_Stack'. (207) (SQLExecDirectW); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'Set_Dimension_Stack'. (207); [42S22] [Microsoft][ODBC SQL Server Driver][SQL Server]Invalid column name 'P1_Attractive_1'. (207); [42S22] [Microsoft]

In [ ]:
import pandas as pd
import json
from imblearn.over_sampling import SMOTE
from collections import Counter
import numpy as np

def BIN_KPOV(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV):
    # Call the summary_describe function
    data = fetch_data(model, line, start, end, selecteKPOV, selecteKPIV)

    # Modify the selected column as needed
    data[selecteKPOV] = pd.cut(
        data[selecteKPOV],
        bins=[-np.inf, float(minKPOV), float(maxKPOV), np.inf],
        labels=['fail_low', 'Pass', 'fail_high']
    )

    # Replace 'fail_low' and 'fail_high' with 'fail'
    data[selecteKPOV].replace(['fail_low', 'fail_high'], 'fail', inplace=True)

    # Count the occurrences of different values in the original data
    count_before_smote = dict(Counter(data[selecteKPOV]))

    k = 3
    X = data.loc[:, data.columns != selecteKPOV]  # Use selecteKPOV here
    y = data[selecteKPOV]  # Use selecteKPOV here

    # Sampling strategy
    sm = SMOTE(sampling_strategy='minority', k_neighbors=k, random_state=100)
    X_res, y_res = sm.fit_resample(X, y)

    # Concatenate the resampled data
    datasets = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)

    # Replace 'fail_low' and 'fail_high' with 'fail' in the resampled data
    datasets[selecteKPOV].replace(['fail_low', 'fail_high'], 'fail', inplace=True)

    # Count the occurrences of different values in the resampled data
    count_after_smote = dict(Counter(datasets[selecteKPOV]))

    # Initialize dictionaries to store AVG and Std for each selecteKPIV
    response_data = {
    "data": response_data_json,
    "count_before_smote": count_before_smote,
    "count_after_smote": count_after_smote
    }
    

    # Iterate over each KPIV in selecteKPIV

    return datasets.dropna(), response_data_json,response_data

# Call BIN_KPOV_KPIV with the provided parameters and preprocessed_data
result_datasets, response_data_json, response_data = BIN_KPOV(
    'LONGSP', '3-6', '2023-08-01', '2023-09-14', 'Projection1',
    ['Parallelism_Stack', 'Set_Dimension_Stack','P1_Attractive_1'], 0.4648, 0.5664
)



In [ ]:
def BIN_KPIV(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV):
    data = fetch_data(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV)
    response_data = {}   
    for KPIV in selecteKPIV:
    # Call summary_describe to get summary information for the current KPIV
        datasets_bin = data_bin(model)
        
    LCL_3 = datasets_bin[KPIV]['LCL_-3']
    LCL_2 = datasets_bin[KPIV]['LCL_-2']
    LCL_1 = datasets_bin[KPIV]['LCL_-1']
    CL = datasets_bin[KPIV]['CL']
    UCL_1 = datasets_bin[KPIV]['UCL_1']
    UCL_2 = datasets_bin[KPIV]['UCL_2']
    UCL_3 = datasets_bin[KPIV]['UCL_3']

    # Print the statistics for the current KPIV
    print(f"{KPIV} Statistics:")
    print(f"3LCL: {LCL_3}")
    print(f"2LCL: {LCL_2}")
    print(f"LCL: {LCL_1}")
    print(f"CL: {CL}")
    print(f"UCL: {UCL_1}")
    print(f"2UCL: {UCL_2}")
    print(f"3UCL: {UCL_3}")

    # Modify the corresponding column in the DataFrame
    column_name = f'{KPIV}'  # Use KPIV as the new column name
    data[column_name] = pd.cut(
        pd.to_numeric(data[column_name], errors='coerce'),
        bins=[-np.inf, LCL_3, LCL_2, LCL_1, CL, UCL_1, UCL_2, UCL_3, np.inf],
        labels=['-3', '-2', '-1', '0', '1', '2', '3', '4'],
        duplicates='drop'
    )

    # Store the statistics in response_data
    response_data[KPIV] = {
        "3LCL": LCL_3,
        "2LCL": LCL_2,
        "LCL": LCL_1,
        "CL": CL,
        "UCL": UCL_1,
        "2UCL": UCL_2,
        "3UCL": UCL_3
    }
    

    print(response_data)

In [ ]:
import pandas as pd
import json
from imblearn.over_sampling import SMOTE
from collections import Counter
import numpy as np

def BIN_KPOV_KPIV(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV):
    # Call the summary_describe function
    data = fetch_data(model, line, start, end, selecteKPOV, selecteKPIV)

    # Modify the selected column as needed
    data[selecteKPOV] = pd.cut(
        data[selecteKPOV],
        bins=[-np.inf, float(minKPOV), float(maxKPOV), np.inf],
        labels=['fail_low', 'Pass', 'fail_high']
    )

    # Replace 'fail_low' and 'fail_high' with 'fail'
    data[selecteKPOV].replace(['fail_low', 'fail_high'], 'fail', inplace=True)

    # Count the occurrences of different values in the original data
    count_before_smote = dict(Counter(data[selecteKPOV]))

    k = 3
    X = data.loc[:, data.columns != selecteKPOV]  # Use selecteKPOV here
    y = data[selecteKPOV]  # Use selecteKPOV here

    # Sampling strategy
    sm = SMOTE(sampling_strategy='minority', k_neighbors=k, random_state=100)
    X_res, y_res = sm.fit_resample(X, y)

    # Concatenate the resampled data
    datasets = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)

    # Replace 'fail_low' and 'fail_high' with 'fail' in the resampled data
    datasets[selecteKPOV].replace(['fail_low', 'fail_high'], 'fail', inplace=True)

    # Count the occurrences of different values in the resampled data
    count_after_smote = dict(Counter(datasets[selecteKPOV]))

    # Initialize dictionaries to store AVG and Std for each selecteKPIV
    AVG_dict = {}
    Std_dict = {}
    response_data = {}

    # Iterate over each KPIV in selecteKPIV
    for KPIV in selecteKPIV:
        # Call summary_describe to get summary information for the current KPIV
        summary_json = json.loads(summary_describe(model, line, start, end, selecteKPOV, KPIV))

        
        # Extract 'mean' and 'std' for the current KPIV
        AVG = summary_json[KPIV]['mean']
        Std = summary_json[KPIV]['std']

        # Store AVG and Std in dictionaries
        AVG_dict[KPIV] = AVG
        Std_dict[KPIV] = Std

        # Print the statistics for the current KPIV
        print(f"{KPIV} Statistics:")
        print(f"AVG: {AVG}")
        print(f"Std: {Std}")
        print(f"3LCL: {AVG - 3 * Std}")
        print(f"2LCL: {AVG - 2 * Std}")
        print(f"LCL: {AVG - 1 * Std}")
        print(f"CL: {AVG}")
        print(f"UCL: {AVG + 1 * Std}")
        print(f"2UCL: {AVG + 2 * Std}")
        print(f"3UCL: {AVG + 3 * Std}")

        # Modify the corresponding column in the DataFrame
        column_name = f'{KPIV}'  # Use KPIV as the new column name
        data[column_name] = pd.cut(
            pd.to_numeric(data[column_name], errors='coerce'),
            bins=[-np.inf, AVG - 3 * Std, AVG - 2 * Std, AVG - Std, AVG, AVG + Std, AVG + 2 * Std, AVG + 3 * Std, np.inf],
            labels=['-3', '-2', '-1', '0', '1', '2', '3', '4'],
            duplicates='drop'
        )

        # Store the statistics in response_data
        response_data[KPIV] = {
            "AVG": AVG,
            "Std": Std,
            "3LCL": AVG - 3 * Std,
            "2LCL": AVG - 2 * Std,
            "LCL": AVG - 1 * Std,
            "CL": AVG,
            "UCL": AVG + 1 * Std,
            "2UCL": AVG + 2 * Std,
            "3UCL": AVG + 3 * Std
        }

    # Convert response_data to JSON
    response_data_json = json.dumps(response_data)
    
    response_data = {
    "data": response_data_json,
    "count_before_smote": count_before_smote,
    "count_after_smote": count_after_smote
    }

    return datasets.dropna(), response_data_json,response_data

# Call BIN_KPOV_KPIV with the provided parameters and preprocessed_data
result_datasets, response_data_json, response_data = BIN_KPOV(
    'LONGSP', '3-6', '2023-08-01', '2023-09-14', 'Projection1',
    ['Parallelism_Stack', 'Set_Dimension_Stack','P1_Attractive_1'], 0.4648, 0.5664
)



In [ ]:
def BIN_KPOV(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV):
    # Call the summary_describe function
    data = fetch_data(model, line, start, end, selecteKPOV, selecteKPIV)

    # Modify the selected column as needed
    data[selecteKPOV] = pd.cut(
        data[selecteKPOV],
        bins=[-np.inf, float(minKPOV), float(maxKPOV), np.inf],
        labels=['fail_low', 'Pass', 'fail_high']
    )

    # Replace 'fail_low' and 'fail_high' with 'fail'
    data[selecteKPOV].replace(['fail_low', 'fail_high'], 'fail', inplace=True)

    # Count the occurrences of different values in the original data
    count_before_smote = dict(Counter(data[selecteKPOV]))

    k = 3
    X = data.loc[:, data.columns != selecteKPOV]  # Use selecteKPOV here
    y = data[selecteKPOV]  # Use selecteKPOV here

    # Sampling strategy
    sm = SMOTE(sampling_strategy='minority', k_neighbors=k, random_state=100)
    X_res, y_res = sm.fit_resample(X, y)

    # Concatenate the resampled data
    datasets = pd.concat([pd.DataFrame(X_res), pd.DataFrame(y_res)], axis=1)

    # Replace 'fail_low' and 'fail_high' with 'fail' in the resampled data
    datasets[selecteKPOV].replace(['fail_low', 'fail_high'], 'fail', inplace=True)

    # Count the occurrences of different values in the resampled data
    count_after_smote = dict(Counter(datasets[selecteKPOV]))

    # Convert the DataFrame to a JSON object
    json_data = datasets.to_json(orient='records')

    # Create a dictionary to include the count results in the JSON response
    response_data = {
        "data": json_data,
        "count_before_smote": count_before_smote,
        "count_after_smote": count_after_smote
      
    }
    return response_data,datasets

In [ ]:
def BIN_KPIV(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV):
    # Call BIN_KPOV to get the initial data
    initial_data = BIN_KPOV(model, line, start, end, selecteKPOV, selecteKPIV, minKPOV, maxKPOV)

    # Convert the JSON data from BIN_KPOV back to a DataFrame
    data_df = pd.read_json(initial_data["data"])

    # Continue with your BIN_KPIV logic to further process data_df

    # Initialize dictionaries to store AVG and Std for each selecteKPIV
    AVG_dict = {}
    Std_dict = {}
    
    # Iterate over each KPIV in selecteKPIV
    for KPIV in selecteKPIV:
        # Call summary_describe to get summary information for the current KPIV
        summary_json = json.loads(data_bin(model, line, start, end, selecteKPOV, KPIV))
        
        # Extract 'mean' and 'std' for the current KPIV
        AVG = summary_json[KPIV]['mean']
        Std = summary_json[KPIV]['std']
        
        # Store AVG and Std in dictionaries
        AVG_dict[KPIV] = AVG  
        Std_dict[KPIV] = Std
    
        print(AVG_dict[KPIV])
        print(Std_dict[KPIV])

        KPIV_3LCL = AVG - 3 * Std
        KPIV_2LCL = AVG - 2 * Std
        KPIV_LCL = AVG - 1 * Std
        KPIV_CL = AVG 
        KPIV_UCL = AVG + 1 * Std
        KPIV_2UCL = AVG + 2 * Std
        KPIV_3UCL = AVG + 3 * Std
 
        # Modify the corresponding 'Datum_probe' column
        column_name = f'{KPIV}'  # ใช้ชื่อ KPIV เป็นชื่อคอลัมน์ใหม่
        data_df[column_name] = pd.cut(
        pd.to_numeric(data_df[column_name], errors='coerce'),  # แปลงค่าในคอลัมน์เป็นตัวเลขและจัดการกับข้อผิดพลาด
        bins=[-np.inf, KPIV_3LCL, KPIV_2LCL, KPIV_LCL, KPIV_CL, KPIV_UCL, KPIV_2UCL, KPIV_3UCL, np.inf],  # กำหนดขอบของช่วง
        labels=['-4','-3','-2','-1','1','2','3','4'],  # กำหนดรหัสสำหรับแต่ละช่วง
        duplicates='drop'  # ตัวเลือกที่ระบุว่าจะลบข้อมูลที่ซ้ำกัน
    

    )
   
    return data_df.dropna()

result_datasets = BIN_KPIV(
    'LONGSP', '3-6', '2023-08-01', '2023-09-14', 'Projection1', 
    ['Datum_probe', 'Max_force'], 0.4648, 0.5664
)
